In [3]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import progressbar
import glob
import re
from urllib.parse import urlparse
import csv

import newspaper
from newspaper import Article

#from web_crawler_spyder import sends_spiders
#from web_crawler_spyder import run_spider
from DMOZ_helper import *
from similarSites_helper import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# get domains with categories

In [4]:
links_news = get_links('http://dmoztools.net/News', False)
links_news

['http://feeds.abcnews.com/abcnews/topstories',
 'http://abcnews.go.com/',
 'http://www.alarabiya.net/',
 'http://www.aljazeera.com/',
 'http://www.aol.com/news/',
 'http://hosted.ap.org/lineups/TOPHEADS-rss_2.0.xml?SITE=RANDOM&SECTION=HOME',
 'http://hosted.ap.org/dynamic/fronts/HOME?SITE=AP&SECTION=HOME',
 'http://www.bbc.co.uk/news/',
 'http://feeds.bbci.co.uk/news/rss.xml',
 'http://feeds.bbci.co.uk/news/rss.xml?edition=int',
 'http://www.cbc.ca/news/',
 'http://rss.cbc.ca/lineup/topstories.xml',
 'http://cbsnews.com/',
 'http://feeds.cbsnews.com/CBSNewsMain?format=xml',
 'http://www.csmonitor.com/',
 'http://www.csmonitor.com/rss/top.rss',
 'http://www.cnn.com/',
 'http://rss.cnn.com/rss/cnn_latest.rss',
 'http://rss.cnn.com/rss/cnn_topstories.rss',
 'http://theconversation.com/',
 'http://www.ctvnews.ca/',
 'http://www.ctvnews.ca/rss/ctvnews-ca-top-stories-public-rss-1.822009',
 'http://www.dw.com/',
 'http://www.economist.com/',
 'http://www.euronews.com/',
 'http://www.foxnews.

In [5]:
def get_sec(time_str):
    if len(time_str) >= 8:
        h, m, s = str(time_str).split(':')
        return int(h) * 3600 + int(m) * 60 + int(s)
    else:
        return ''


def get_percentage(perc_str):
    if '%' in perc_str:
        return (float(str(perc_str).strip('%')) / 100)
    else:
        return 0


def build_webistes_df(domain_list):

    # rename columns correctly
    frame = pd.DataFrame(columns=[
        'website', 'category', 'subcategory', 'rank', 'avg_visit_duration',
        'pages_per_visit', 'bounce_rate'
    ])
    with progressbar.ProgressBar(max_value=len(domain_list)) as bar:
        for i, d in enumerate(domain_list):
            frame.loc[i] = get_similarsite_features(d)
            bar.update(i)

    # rank removing -
    frame['rank'] = frame['rank'].apply(lambda x: x.replace('-', ''))

    # visit duration in sec
    frame.avg_visit_duration = frame.avg_visit_duration.map(get_sec)

    # page/visit
    frame.pages_per_visit = frame.pages_per_visit.apply(
        lambda x: x.strip('/visit').strip('-'))

    # percentage string to float/100
    frame['bounce_rate'] = frame['bounce_rate'].map(get_percentage)

    # drop duplicate, for now they have the same categories so it's ok to do so
    frame = frame.drop_duplicates(['website'])

    return frame

In [6]:
build_webistes_df(links_news).head()

 73% (47 of 64) |##################       | Elapsed Time: 0:00:41 ETA:  0:00:12

no soup for http://feeds.skynews.com/sky-news/rss/home/rss.xml


100% (64 of 64) |#########################| Elapsed Time: 0:00:55 Time: 0:00:55


,website,category,subcategory,rank,avg_visit_duration,pages_per_visit,bounce_rate
0,http://feeds.abcnews.com/abcnews/topstories,,rss feeds,,,,0.00
1,http://abcnews.go.com/,News And Media,news,,,,0.00
2,http://www.alarabiya.net/,News And Media,صحف,1782,210,3.01,0.43
3,http://www.aljazeera.com/,News And Media,news,1787,539,2.8,0.59
4,http://www.aol.com/news/,Internet And Telecom,aol,91,408,5.22,0.35


# Fetch new fdkl jfélds jfédsa

In [ ]:
MAX_ARTICLES_PER_DOMAIN = 2
PATH = '../Datasets/articles/'


X = pd.DataFrame(columns=[
    'has_author', 'authors', 'has_images', 'num_images', 'keywords', 'lang',
    'has_movie', 'publish_date', 'tags', 'text', 'title', 'url', 'meta_data'
])

    
def save_article_feature(article):
    try:
        article.download()
        if article.title or article.html:
            article.parse()
            article.nlp()
            new_row = [
                len(article.authors) > 0, article.authors,
                len(article.images) > 0,
                len(article.images), article.keywords, article.meta_lang,
                len(article.movies) > 0, article.publish_date, article.tags,
                article.text, article.title, article.url, article.meta_data]
            with open(PATH + 'articles_en.csv', 'a+', newline='') as csvfile:
                spamwriter = csv.writer(csvfile, delimiter=',')
                spamwriter.writerow(new_row)
    except:
        print('oupsi not a good article')


def add_article_feature(df, article):
    try:
        article.download()
        if article.title or article.html:
            article.parse()
            article.nlp()
            new_row = [
                len(article.authors) > 0, article.authors,
                len(article.images) > 0,
                len(article.images), article.keywords, article.meta_lang,
                len(article.movies) > 0, article.publish_date, article.tags,
                article.text, article.title, article.url, article.meta_data]
            df.loc[len(df)] = new_row
            print(df)
    except:
        print('oupsi not a good article')
    return df

In [31]:
def articles_from_domain(domain, number=100):
    papers = newspaper.build(domain, language='en')
    #max_art = min(number, len(papers.articles))
    return papers.articles[:number]

with progressbar.ProgressBar(max_value=len(links_news)) as bar:
    for i,domain in enumerate(links_news):

        temp_arts = articles_from_domain(domain, MAX_ARTICLES_PER_DOMAIN)

        if len(temp_arts) >= 1:
            for art in temp_arts:
                save_article_feature(art)
        else:
            run_spider(domain, MAX_ARTICLES_PER_DOMAIN)
        bar.update(i)

 15% (10 of 64) |###                      | Elapsed Time: 0:00:59 ETA:  0:04:26

oupsi not a good article


 35% (23 of 64) |########                 | Elapsed Time: 0:02:03 ETA:  0:02:52

oupsi not a good article


 37% (24 of 64) |#########                | Elapsed Time: 0:02:13 ETA:  0:03:07

...

 81% (52 of 64) |####################     | Elapsed Time: 0:06:55 ETA:  0:01:00

oupsi not a good article


 90% (58 of 64) |######################   | Elapsed Time: 0:07:26 ETA:  0:00:33

oupsi not a good article


100% (64 of 64) |#########################| Elapsed Time: 0:07:52 Time: 0:07:52


## fetch articles when possible

In [4]:
def articles_from_domain(domain, number=100):
    papers = newspaper.build(domain, language='en')
    return papers.articles[:number]

In [5]:
MAX_ARTICLES_PER_DOMAIN = 100

domain_with_no_articles = []
arts = []

with progressbar.ProgressBar(max_value=len(frame.website)) as bar:
    for i, d in enumerate(frame.website):
        bar.update(i)
        full_website_name = 'http://www.' + d
        temp_arts = articles_from_domain(full_website_name,
                                         MAX_ARTICLES_PER_DOMAIN)
        arts = arts + temp_arts
        if len(temp_arts) == 0:
            domain_with_no_articles.append(full_website_name)

  2% ( 11 of 435) |#                                                               | Elapsed Time: 0:00:42 ETA: 0:21:02

[Source parse ERR] http://www.se.com.br


  3% ( 15 of 435) |##                                                              | Elapsed Time: 0:00:54 ETA: 0:23:12

[Source parse ERR] http://www.vegetarisch.de


  9% ( 41 of 435) |######                                                          | Elapsed Time: 0:02:24 ETA: 0:16:01

...

[Source parse ERR] http://www.isky.am


 94% (413 of 435) |############################################################    | Elapsed Time: 0:54:37 ETA: 0:02:47

[Source parse ERR] http://www.redalyc.org


100% (435 of 435) |###############################################################| Elapsed Time: 0:56:12 Time: 0:56:12


LocationValueError: No host specified.

In [6]:
domain_with_no_articles

['http://www.happycow.net',
 'http://www.irrawaddy.com',
 'http://www.veganricha.com',
 'http://www.froh.de',
 'http://www.veggieboards.com',
 'http://www.rawdd.com',
 'http://www.stadionowioprawcy.net',
 'http://www.vegansociety.com',
 'http://www.vebu.de',
 'http://www.chooseveg.com',
 'http://www.farmfreshtoyou.com',
 'http://www.se.com.br',
 'http://www.holycowvegan.net',
 'http://www.vegnews.com',
 'http://www.msrawytop.com',
 'http://www.vegetarisch.de',
 'http://www.veganuary.com',
 'http://www.vegan.com',
 'http://www.ilovevegan.com',
 'http://www.vegancoach.com',
 'http://www.gesund.de',
 'http://www.suiis.com',
 'http://www.biserawalpindi.edu.pk',
 'http://www.vegetarianismo.net',
 'http://www.brandnewvegan.com',
 'http://www.vegweb.com',
 'http://www.veganessentials.com',
 'http://www.veggiecommunity.org',
 'http://www.followyourheart.com',
 'http://www.mania.com',
 'http://www.veganbodybuilding.com',
 'http://www.raws.org',
 'http://www.sunwarrior.com',
 'http://www.veganfo

In [7]:
len(arts)

4667

## Find pages for when needed

In [ ]:
sends_spiders(domain_with_no_articles, MAX_ARTICLES_PER_DOMAIN)

run_spider(do, 5)  # instead

In [8]:
path = '../Datasets/crawled_articles_url'

articles_to_get = dataframe_from_csvs(path)

In [9]:
for d in articles_to_get.url:
    try:
        temp_art = Article(d)
        arts.append(temp_art)
    except:
        pass

In [ ]:
t = arts[0]
t.download()
t.parse()
t.nlp()

In [ ]:
t.html

## feature extraction

In [10]:
X = pd.DataFrame(columns=[
    'has_author', 'authors', 'has_images', 'num_images', 'keywords', 'lang',
    'has_movie', 'publish_date', 'tags', 'text', 'title', 'url', 'meta_data',
    'html'
])

with progressbar.ProgressBar(max_value=len(arts)) as bar:
    for i, a in enumerate(arts):
        bar.update(i)
        try:
            a.download()
            if a.title or a.html:
                a.parse()
                a.nlp()
                X.loc[i] = [
                    len(a.authors) > 0, a.authors,
                    len(a.images) > 0,
                    len(a.images), a.keywords, a.meta_lang,
                    len(a.movies) > 0, a.publish_date, a.tags, a.text, a.title,
                    a.url, a.meta_data, a.html
                ]
        except:
            pass

#X = pd.merge(X, articles_to_get, how='left', left_on='url', right_on='url')

  0% (    0 of 14312) |                                                          | Elapsed Time: 0:00:00 ETA:  --:--:--

Article `download()` failed with 404 Client Error: Not Found for url: https://vegetarian.ru/bitrix/urlrewrite.php on URL http://www.vegetarian.ru/news/velosipedist-iz-indii-vdokhnovlyaet-lyudey-stat-veganami.html


  0% (    1 of 14312) |                                                            | Elapsed Time: 0:00:01 ETA: 4:54:00

Article `download()` failed with 404 Client Error: Not Found for url: https://vegetarian.ru/bitrix/urlrewrite.php on URL http://www.vegetarian.ru/news/rossiyanka-olga-mironenko-stala-laureatom-ekologicheskoy-premii-green-talents.html


  0% (    2 of 14312) |                                                            | Elapsed Time: 0:00:02 ETA: 4:20:19

...

Article `download()` failed with 404 Client Error: Not Found for url: http://video.wanfangdata.com.cn/%09%09%09/v/play/SA030101258.html%0D%0A%09 on URL http://video.wanfangdata.com.cn/			/v/play/SA030101258.html
	


 31% ( 4564 of 14312) |###################                                         | Elapsed Time: 2:06:12 ETA: 1:34:03

Article `download()` failed with 404 Client Error: Not Found for url: http://video.wanfangdata.com.cn/%09%09%09/v/play/SG160215338.html%0D%0A%09 on URL http://video.wanfangdata.com.cn/			/v/play/SG160215338.html
	


 33% ( 4799 of 14312) |###################                                        | Elapsed Time: 2:33:47 ETA: 16:45:57C:\Users\Leonovo\Anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 49 bytes but only got 36. Skipping tag 305
  "Skipping tag %s" % (size, len(data), tag))
C:\Users\Leonovo\Anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 49 bytes but only got 0. Skipping tag 305
  "Skipping tag %s" % (size, len(data), tag))
C:\Users\Leonovo\Anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 98 bytes but only got 0. Skipping tag 37500
  "Skipping tag %s" % (size, len(data), tag))
C:\Users\Leonovo\Anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 4980 bytes but only got 266. Skipping tag 37510
  "Skipping tag %s" % (size, len(d

 93% (13315 of 14312) |######################################################     | Elapsed Time: 10:25:06 ETA: 0:27:39C:\Users\Leonovo\Anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 4834 bytes but only got 0. Skipping tag 37500
  "Skipping tag %s" % (size, len(data), tag))
C:\Users\Leonovo\Anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 264 bytes but only got 224. Skipping tag 37510
  "Skipping tag %s" % (size, len(data), tag))
C:\Users\Leonovo\Anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 33 bytes but only got 0. Skipping tag 42016
  "Skipping tag %s" % (size, len(data), tag))
C:\Users\Leonovo\Anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 32 bytes but only got 0. Skipping tag 42032
  "Skipping tag %s" % (size, l

In [11]:
def get_base_domain(url):
    parsed_uri = urlparse(url)
    domain = '{uri.scheme}://{uri.netloc}/'.format(uri=parsed_uri)
    return domain


X = pd.merge(X, articles_to_get, how='left', left_on='url', right_on='url')
X.base_url = X[X.base_url.isnull()].base_url = X.url.apply(
    lambda x: get_base_domain(x))

In [12]:
X

,has_author,authors,has_images,num_images,keywords,lang,has_movie,publish_date,tags,text,title,url,meta_data,html,base_url
0,True,[The Flaming Vegan Crew],True,29,"[leather, flaming, perfect, staff, vegan, bag,...",,False,NaT,{},If you’re in need of the perfect holiday gifts...,The Flaming Vegan Staff Pick: Luca Chiara,https://www.theflamingvegan.com/view-post/The-...,"{'copyright': 'Copyright 2010-2013 GoodBlogs, ...","<!DOCTYPE HTML>\n<html>\n<head prefix=""og: htt...",https://www.theflamingvegan.com/
1,True,[The Flaming Vegan Crew],True,29,"[leather, flaming, perfect, staff, vegan, bag,...",,False,NaT,{},If you’re in need of the perfect holiday gifts...,The Flaming Vegan Staff Pick: Luca Chiara,https://www.theflamingvegan.com/view-post/The-...,"{'copyright': 'Copyright 2010-2013 GoodBlogs, ...","<!DOCTYPE HTML>\n<html>\n<head prefix=""og: htt...",https://www.theflamingvegan.com/
2,False,[],True,31,[张可盈出道竟不见母亲张凯丽韩红谭维维众大咖护航],,False,NaT,{},,张可盈出道竟不见母亲张凯丽！韩红谭维维众大咖护航,http://www.cntvan.com/2017/1117/102317.html,"{'keywords': '张可盈,张凯丽,韩红,谭维维', 'description': ...","<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0 T...",http://www.cntvan.com/
3,False,[],True,4,"[update, guide, gloves, leather, blog, resourc...",en,False,2010-12-27 00:00:00,{},"Posted on December 27, 2010 by The VRG Blog Ed...",The Vegetarian Resource Group Blog,http://www.vrg.org/blog/2010/12/27/updated-veg...,{'generator': 'WordPress 4.6.8'},"<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0 T...",http://www.vrg.org/
4,False,[],True,33,"[cantinho, vegana, vegetariano, farofa, soja, ...",,False,None,"{PVT ou PTS, Vegana, Farofas, Farinha de Mandi...",,Cantinho Vegetariano: Farofa de Soja Crocante ...,http://www.cantinhovegetariano.com.br/2017/11/...,"{'viewport': 'width=1100', 'generator': 'blogg...",<!DOCTYPE html>\n<html class='v2' dir='ltr' xm...,http://www.cantinhovegetariano.com.br/
5,False,[],True,45,"[ou, cantinho, pts, vegetariano, que, é, pvt]",,False,None,"{PVT ou PTS, ABC Culinário}",,Cantinho Vegetariano: O que é PTS ou PVT?,http://www.cantinhovegetariano.com.br/2007/06/...,"{'viewport': 'width=1100', 'generator': 'blogg...",<!DOCTYPE html>\n<html class='v2' dir='ltr' xm...,http://www.cantinhovegetariano.com.br/
6,False,[],True,33,"[cantinho, vegana, vegetariano, farofa, soja, ...",,False,None,"{PVT ou PTS, Vegana, Farofas, Farinha de Mandi...",,Cantinho Vegetariano: Farofa de Soja Crocante ...,http://www.cantinhovegetariano.com.br/2017/11/...,"{'viewport': 'width=1100', 'generator': 'blogg...",<!DOCTYPE html>\n<html class='v2' dir='ltr' xm...,http://www.cantinhovegetariano.com.br/
7,False,[],True,33,"[cantinho, vegana, vegetariano, farofa, soja, ...",,False,None,"{PVT ou PTS, Vegana, Farofas, Farinha de Mandi...",,Cantinho Vegetariano: Farofa de Soja Crocante ...,http://www.cantinhovegetariano.com.br/2017/11/...,"{'viewport': 'width=1100', 'generator': 'blogg...",<!DOCTYPE html>\n<html class='v2' dir='ltr' xm...,http://www.cantinhovegetariano.com.br/
8,False,[],True,108,"[ヘアサロンcuaura, salonのヘアカタログホットペッパービューティー, ジェントル...",,False,2025-09-19 00:00:00,{},,ジェントルマン×サイドパート：L007450079｜クオーラ ヘアサロン(Cuaura HA...,https://beauty.hotpepper.jp/slnH000190925/styl...,{'copyright': 'Copyright © Recruit Lifestyle C...,"<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0 T...",https://beauty.hotpepper.jp/
9,False,[],True,108,"[salonのヘアカタログホットペッパービューティー, ヘアサロンcuaura, ワイルドナ...",,False,2025-09-19 00:00:00,{},,ワイルドナチュラルショート：L007450072｜クオーラ ヘアサロン(Cuaura HAI...,https://beauty.hotpepper.jp/slnH000190925/styl...,{'copyright': 'Copyright © Recruit Lifestyle C...,"<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0 T...",https://beauty.hotpepper.jp/


In [13]:
# load csv articles obtained with this list of websites
X.to_csv(
    '../Datasets/articles_from_Article_with_html.csv',
    sep='\t',
    encoding='utf-8')

## 

In [56]:
k = pd.DataFrame.from_dict(
    X[X.lang == 'en'].meta_data.values[500], orient='index').loc['og'].values

In [61]:
pd.DataFrame.from_records(k)

,description,image,locale,site_name,title,type,updated_time,url
0,Naidu also urged the media to stay away from p...,{'identifier': 'http://images.indianexpress.co...,en_US,The Indian Express,"On National Press Day, V-P Venkaiah Naidu warn...",article,"November 17, 2017 4:27 am",http://indianexpress.com/article/india/on-nati...


In [70]:
for x in X.meta_data.values:
    print(x['og']['type'])

article
article


KeyError: 'type'